<a href="https://colab.research.google.com/github/nicholasketo/Stock-Market-Value-Prediction/blob/main/Quant_momentum_investing_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install xlsxwriter

In [2]:
import numpy as np
import pandas as pd
import requests as rq
import math
from scipy import stats
import xlsxwriter


In [3]:
url = 'https://raw.githubusercontent.com/nicholasketo/Stock-Market-Value-Prediction/main/sp_500_stocks.csv'
stocks = pd.read_csv(url)
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [4]:
IEX_CLOUD_API_TOKEN = 'sk_30a7d5719bf14aa584faa9c117f2988f'

In [5]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
try:
    response = rq.get(api_url)
    response.raise_for_status()  # Check for any HTTP errors
    data = response.json()  # Try to parse the response as JSON

    # If you want to print the data
    print(data)

except rq.exceptions.RequestException as e:
    print(f"Request failed: {e}")
except ValueError as e:
    print(f"ValueError: {e}")

{'companyName': 'Apple Inc', 'marketcap': 2827801368640, 'week52high': 197.7, 'week52low': 123.48, 'week52highSplitAdjustOnly': 198.23, 'week52lowSplitAdjustOnly': 124.17, 'week52change': 0.31235149250930383, 'sharesOutstanding': 15552752000, 'float': 0, 'avg10Volume': 63492450, 'avg30Volume': 58016722, 'day200MovingAvg': 179.3, 'day50MovingAvg': 174.76, 'employees': 164000, 'ttmEPS': 6.13, 'ttmDividendRate': 0.7084164673375632, 'dividendYield': 0.00389625160784052, 'nextDividendDate': '2023-11-10', 'exDividendDate': '2023-08-11', 'nextEarningsDate': '2024-02-01', 'peRatio': 29.154094217640083, 'beta': 0, 'maxChangePercent': 69.87946358958365, 'year5ChangePercent': 2.626391160397304, 'year2ChangePercent': 0.2171838347918296, 'year1ChangePercent': 0.3178313115038076, 'ytdChangePercent': 0.4072363308762295, 'month6ChangePercent': 0.05181499116931243, 'month3ChangePercent': 0.01935324721923215, 'month1ChangePercent': 0.02439574060510452, 'day30ChangePercent': 0.02439574060510452, 'day5Cha

In [6]:
data['year1ChangePercent']

0.3178313115038076

In [7]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [8]:

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol,
                           data[symbol]['quote'].get('latestPrice', 'N/A'),
                           data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                           'N/A'],
                          index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

final_dataframe

<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: ABC


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: ANTM


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

...

Data not found for symbol: VIAC


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

Data not found for symbol: WLTW


<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-8-eb0fa1bdae93>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,109.39,-0.193502,N/A
1,AAL,12.01,-0.179469,N/A
2,AAP,57.64,-0.670976,N/A
3,AAPL,182.89,0.317831,N/A
4,ABBV,142.03,-0.001331,N/A
...,...,...,...,...
491,YUM,125.61,0.043362,N/A
492,ZBH,104.86,-0.007333,N/A
493,ZBRA,206.44,-0.119817,N/A
494,ZION,33.76,-0.285414,N/A


In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
  final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/ final_dataframe.loc[i, 'Price'])

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,109.39,-0.193502,18
1,AAL,12.01,-0.179469,167
2,AAP,57.64,-0.670976,34
3,AAPL,182.89,0.317831,11
4,ABBV,142.03,-0.001331,14
...,...,...,...,...
491,YUM,125.61,0.043362,16
492,ZBH,104.86,-0.007333,19
493,ZBRA,206.44,-0.119817,9
494,ZION,33.76,-0.285414,59


In [38]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'

]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = rq.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data and 'quote' in data[symbol] and 'stats' in data[symbol]:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series([
                   symbol,
                   data[symbol]['quote'].get('latestPrice', 'N/A'),
                  'N/A',
                   data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month6ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month3ChangePercent', 'N/A'),
                  'N/A',
                  data[symbol]['stats'].get('month1ChangePercent', 'N/A'),
                  'N/A',
                  'N/A'
                ],
                index=hqm_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

hqm_dataframe



<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: ABC
Data not found for symbol: ANTM


<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: DISCA
Data not found for symbol: FBHS


<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

...

Data not found for symbol: RE


<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Data not found for symbol: VIAC
Data not found for symbol: WLTW


<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-38-dc3d107cf68d>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,109.39,N/A,-0.193502,N/A,-0.167484,N/A,-0.124473,N/A,-0.000904,N/A,N/A
1,AAL,12.01,N/A,-0.179469,N/A,-0.152848,N/A,-0.265166,N/A,-0.079154,N/A,N/A
2,AAP,57.64,N/A,-0.670976,N/A,-0.524326,N/A,-0.177194,N/A,0.128412,N/A,N/A
3,AAPL,182.89,N/A,0.317831,N/A,0.051815,N/A,0.019353,N/A,0.024396,N/A,N/A
4,ABBV,142.03,N/A,-0.001331,N/A,-0.019289,N/A,-0.044882,N/A,-0.031416,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,125.61,N/A,0.043362,N/A,-0.078798,N/A,-0.066924,N/A,0.050393,N/A,N/A
492,ZBH,104.86,N/A,-0.007333,N/A,-0.228942,N/A,-0.154979,N/A,-0.044901,N/A,N/A
493,ZBRA,206.44,N/A,-0.119817,N/A,-0.240175,N/A,-0.161906,N/A,-0.070628,N/A,N/A
494,ZION,33.76,N/A,-0.285414,N/A,0.47734,N/A,-0.095546,N/A,0.012171,N/A,N/A


In [39]:
%pip install stats

In [40]:
import numpy as np
from scipy import stats

time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        data = hqm_dataframe[change_col].dropna()  # Drop rows with missing values

        if data.empty:
            print(f"No data available for {change_col}")
        else:
            score = hqm_dataframe.loc[row, change_col]
            if score is not None:
                hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(data, score) / 100
            else:
                print(f"No valid data available for {change_col}")

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

# Print the entire DataFrame
hqm_dataframe


No valid data available for One-Year Price Return
No valid data available for Six-Month Price Return
No valid data available for Three-Month Price Return
No valid data available for One-Month Price Return
No valid data available for One-Year Price Return
No valid data available for Six-Month Price Return
No valid data available for Three-Month Price Return
No valid data available for One-Month Price Return
No valid data available for One-Year Price Return
No valid data available for Six-Month Price Return
No valid data available for Three-Month Price Return
No valid data available for One-Month Price Return
No valid data available for One-Year Price Return
No valid data available for Six-Month Price Return
No valid data available for Three-Month Price Return
No valid data available for One-Month Price Return
0      0.166667
1      0.180894
2      0.006098
3      0.865854
4      0.449187
         ...   
491    0.550813
492    0.434959
493    0.252033
494    0.065041
495    0.780488
Name

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,109.39,N/A,-0.193502,0.166667,-0.167484,0.182927,-0.124473,0.272358,-0.000904,0.402439,N/A
1,AAL,12.01,N/A,-0.179469,0.180894,-0.152848,0.203252,-0.265166,0.042683,-0.079154,0.101626,N/A
2,AAP,57.64,N/A,-0.670976,0.006098,-0.524326,0.004065,-0.177194,0.128049,0.128412,0.95122,N/A
3,AAPL,182.89,N/A,0.317831,0.865854,0.051815,0.623984,0.019353,0.810976,0.024396,0.581301,N/A
4,ABBV,142.03,N/A,-0.001331,0.449187,-0.019289,0.455285,-0.044882,0.583333,-0.031416,0.256098,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,125.61,N/A,0.043362,0.550813,-0.078798,0.339431,-0.066924,0.487805,0.050393,0.737805,N/A
492,ZBH,104.86,N/A,-0.007333,0.434959,-0.228942,0.089431,-0.154979,0.197154,-0.044901,0.197154,N/A
493,ZBRA,206.44,N/A,-0.119817,0.252033,-0.240175,0.077236,-0.161906,0.176829,-0.070628,0.117886,N/A
494,ZION,33.76,N/A,-0.285414,0.065041,0.47734,0.981707,-0.095546,0.371951,0.012171,0.510163,N/A


In [41]:
import pandas as pd

# Assuming hqm_dataframe and time_periods are defined

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(pd.to_numeric(hqm_dataframe.loc[row, f'{time_period} Return Percentile'], errors='coerce'))
    momentum_percentiles = [0 if pd.isnull(x) else x for x in momentum_percentiles]
    hqm_dataframe.loc[row, 'HQM Score'] = sum(momentum_percentiles) / len(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,109.39,N/A,-0.193502,0.166667,-0.167484,0.182927,-0.124473,0.272358,-0.000904,0.402439,0.256098
1,AAL,12.01,N/A,-0.179469,0.180894,-0.152848,0.203252,-0.265166,0.042683,-0.079154,0.101626,0.132114
2,AAP,57.64,N/A,-0.670976,0.006098,-0.524326,0.004065,-0.177194,0.128049,0.128412,0.95122,0.272358
3,AAPL,182.89,N/A,0.317831,0.865854,0.051815,0.623984,0.019353,0.810976,0.024396,0.581301,0.720528
4,ABBV,142.03,N/A,-0.001331,0.449187,-0.019289,0.455285,-0.044882,0.583333,-0.031416,0.256098,0.435976
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,125.61,N/A,0.043362,0.550813,-0.078798,0.339431,-0.066924,0.487805,0.050393,0.737805,0.528963
492,ZBH,104.86,N/A,-0.007333,0.434959,-0.228942,0.089431,-0.154979,0.197154,-0.044901,0.197154,0.229675
493,ZBRA,206.44,N/A,-0.119817,0.252033,-0.240175,0.077236,-0.161906,0.176829,-0.070628,0.117886,0.155996
494,ZION,33.76,N/A,-0.285414,0.065041,0.47734,0.981707,-0.095546,0.371951,0.012171,0.510163,0.482215


In [46]:
sorted_dataframe= hqm_dataframe.sort_values('HQM Score',ascending = False)
sorted_dataframe = sorted_dataframe[:50]
sorted_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
8,ADBE,585.31,N/A,0.953662,0.993902,0.679679,0.997967,0.104714,0.949187,0.111111,0.928862,0.96748
190,FTI,20.82,N/A,0.741398,0.98374,0.484765,0.985772,0.162522,0.971545,0.102403,0.906504,0.96189
270,LB,79.92,N/A,2.281287,1.0,0.815831,1.0,0.215421,0.985772,0.079271,0.849593,0.958841
204,GPS,13.52,N/A,0.293812,0.843496,0.619159,0.995935,0.310759,0.993902,0.327805,1.0,0.958333
36,ANET,209.60,N/A,0.628121,0.965447,0.540803,0.99187,0.185921,0.977642,0.095763,0.892276,0.956809
333,NOW,625.33,N/A,0.690281,0.977642,0.411937,0.977642,0.106363,0.95122,0.09486,0.890244,0.949187
278,LLY,619.13,N/A,0.661098,0.973577,0.40899,0.97561,0.324031,0.995935,0.06141,0.79065,0.933943
169,EXPE,116.56,N/A,0.317669,0.863821,0.262027,0.926829,0.098242,0.947154,0.174861,0.985772,0.930894
105,COG,22.25,N/A,0.340483,0.884146,0.180873,0.849593,0.255629,0.99187,0.249298,0.99187,0.92937
75,CAH,102.27,N/A,0.302978,0.855691,0.251832,0.916667,0.123371,0.955285,0.145335,0.96748,0.92378


In [47]:
portfolio_input()

Enter the value of your portfolio:1000000


In [49]:
position_size = float(portfolio_size) / len(sorted_dataframe.index)
sorted_dataframe.reset_index(drop=True, inplace= True)
for i in range(0, len(sorted_dataframe)-1):
    sorted_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / sorted_dataframe['Price'][i])
sorted_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ADBE,585.31,34,0.953662,0.993902,0.679679,0.997967,0.104714,0.949187,0.111111,0.928862,0.96748
1,FTI,20.82,960,0.741398,0.98374,0.484765,0.985772,0.162522,0.971545,0.102403,0.906504,0.96189
2,LB,79.92,250,2.281287,1.0,0.815831,1.0,0.215421,0.985772,0.079271,0.849593,0.958841
3,GPS,13.52,1479,0.293812,0.843496,0.619159,0.995935,0.310759,0.993902,0.327805,1.0,0.958333
4,ANET,209.60,95,0.628121,0.965447,0.540803,0.99187,0.185921,0.977642,0.095763,0.892276,0.956809
5,NOW,625.33,31,0.690281,0.977642,0.411937,0.977642,0.106363,0.95122,0.09486,0.890244,0.949187
6,LLY,619.13,32,0.661098,0.973577,0.40899,0.97561,0.324031,0.995935,0.06141,0.79065,0.933943
7,EXPE,116.56,171,0.317669,0.863821,0.262027,0.926829,0.098242,0.947154,0.174861,0.985772,0.930894
8,COG,22.25,898,0.340483,0.884146,0.180873,0.849593,0.255629,0.99187,0.249298,0.99187,0.92937
9,CAH,102.27,195,0.302978,0.855691,0.251832,0.916667,0.123371,0.955285,0.145335,0.96748,0.92378


In [50]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [51]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [52]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [53]:
writer.save()

<ipython-input-53-16d93d5523a4>:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [54]:
from google.colab import files

# Assuming sorted_dataframe is your DataFrame
excel_file_name = 'momentum_investing_strategy.xlsx'

# Export the DataFrame to an Excel file
sorted_dataframe.to_excel(excel_file_name, index=False)

# Download the Excel file
files.download(excel_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>